##### upload csv file

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd
from sklearn import preprocessing

# used for dates
import datetime
from datetime import datetime, date

# Set some pandas options controlling output format
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 90)

# bring in matplotlib for graphics
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read output #1 (repalce space with underscore in all columns)
root = 'data'
msft = pd.read_csv( root + '/' +'1.member_column_cleanup.csv')
#msft[:15]

In [3]:
df = msft.copy()

In [4]:
df.shape

(21344, 112)

In [5]:
# split na from memeber tenure years before binning data
# add df_na back at the end of binning data process
df_na = df[df['Member_Tenure_Years'].isnull()].copy()
df_temp = df[df['Member_Tenure_Years'].notnull()].copy()

In [6]:
# print(df_na.shape)
# df_na.transpose()
# # forget this row since it has not much data in it
# # no need to add it back

In [7]:
df_temp.columns

Index(['Individual_Key', 'Household_Key', 'Member_Flag', 'City', 'State_-_Grouped',
       'ZIP5', 'ZIP9', 'FSV_CMSI_Flag', 'FSV_Credit_Card_Flag',
       'FSV_Deposit_Program_Flag',
       ...
       'SC_Vehicle_Manufacturer_Name', 'SC_Vehicle_Model_Name', 'SVC_Facility_Name',
       'SVC_Facility_Type', 'Total_Cost', 'Tow_Destination_Latitude',
       'Tow_Destination_Longitude', 'Tow_Destination_Name', 'Was_Duplicated',
       'Was_Towed_To_AAR_Referral'],
      dtype='object', length=112)

In [8]:
x = df_temp['Occupation_Group'].dropna()
x.head()

20                     Retired - Known
22    Office Administration - Inferred
23    Office Administration - Inferred
24    Office Administration - Inferred
25    Office Administration - Inferred
Name: Occupation_Group, dtype: object

In [9]:
li = df_temp['Member_Tenure_Years'].unique().tolist()
#sorted(li)

In [10]:
max_value = df_temp['Member_Tenure_Years'].max()
print('max value:', int(max_value))

max value: 98


In [11]:
# Deciles as integer from 1 to 10
df_temp['Member_Tenure_Decile'] = pd.qcut(df_temp['Member_Tenure_Years'],q=10,labels=False) + 1
df_temp['Member_Tenure_Decile'].value_counts().sort_index()

1     2317
2     2114
3     2183
4     2016
5     2162
6     2625
7     2104
8     2892
9     1144
10    1786
Name: Member_Tenure_Decile, dtype: int64

###### The iter() function takes two parameters:

object - object whose iterator has to be created (can be sets, tuples, etc.)

sentinel (optional) - special value that is used to represent the end of a sequence

In [12]:
# # convert list to dictionary
# lst = ['a', 1, 'b', 2, 'c', 3] 
# it=iter(lst)
# res_dct = dict(zip(it, it)) 
# print(res_dct)

In [13]:
# assign memeber tenure tier

category,bins = pd.qcut(df_temp['Member_Tenure_Years'],q=10,retbins=True,labels=False)

bins_lst=[]
for i in range(0,len(bins)):
    if i == 0:
        bins_lst = [1,'0-10']
    elif i+1 < len(bins):
        k = (i+1)
        bins_lst.append(k)
        v = str(int(bins[i])+1) + "-" + str(int(bins[i+1]))
        bins_lst.append(v)
        
#print(bins_lst)
it=iter(bins_lst)
bins_dct = dict(zip(it, it)) 
print(bins_dct)

{1: '0-10', 2: '10-13', 3: '14-18', 4: '19-25', 5: '26-35', 6: '36-46', 7: '47-47', 8: '48-49', 9: '50-55', 10: '56-98'}


In [14]:
# repalce category key with a more describe names by using map function
# add a new column to df_temp
df_temp['Member_Tenure_Tier'] = df_temp['Member_Tenure_Decile'].map(bins_dct)
#df_na['Member_Tenure_Tier'] = np.nan

In [15]:
#df_temp['Member_Tenure_Tier'].unique()[:15]
#df_na['Member_Tenure_Tier']

In [16]:
# spike_cols = [col for col in df_temp.columns if 'Decile' in col]
# print(spike_cols)
# spike_cols = [col for col in df_na.columns if 'Decile' in col]
# print(spike_cols)

In [17]:
# delete Member_Tenure_Decile column from df_temp before merge it back with df_na
# del df_temp['Member_Tenure_Decile']
# spike_cols = [col for col in df_temp.columns if 'Decile' in col]
# print(spike_cols)

In [18]:
# df1 = pd.concat([df_temp, df_na], ignore_index=True, sort=True)
# print("df_temp:", df_temp.shape)
# print("df_na:", df_na.shape)
# print("df1.shape", df1.shape)

In [19]:
df_temp.columns

Index(['Individual_Key', 'Household_Key', 'Member_Flag', 'City', 'State_-_Grouped',
       'ZIP5', 'ZIP9', 'FSV_CMSI_Flag', 'FSV_Credit_Card_Flag',
       'FSV_Deposit_Program_Flag',
       ...
       'SVC_Facility_Name', 'SVC_Facility_Type', 'Total_Cost',
       'Tow_Destination_Latitude', 'Tow_Destination_Longitude', 'Tow_Destination_Name',
       'Was_Duplicated', 'Was_Towed_To_AAR_Referral', 'Member_Tenure_Decile',
       'Member_Tenure_Tier'],
      dtype='object', length=114)

In [20]:
x1 = df_temp['Occupation_Group'].dropna()
x1.head()

20                     Retired - Known
22    Office Administration - Inferred
23    Office Administration - Inferred
24    Office Administration - Inferred
25    Office Administration - Inferred
Name: Occupation_Group, dtype: object

In [21]:
# write the output to data sub-directory
root = 'data'
df_temp.to_csv( root + '/' +'2.member_tenure_tier.csv',index=False)